In [5]:
// Data types and functions
using System.Text.RegularExpressions;

public record struct Room {
    public List<string> Tunnels;
    public int FlowRate;
}

var RoomRegex = new Regex(@"Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? ((?>\w+(?>, )?)+)");
(string, Room) ParseRoom(string line) {
    var matchGroups = RoomRegex.Match(line).Groups;
    return (
        matchGroups[1].Value,
        new Room {
            FlowRate = int.Parse(matchGroups[2].Value),
            Tunnels = matchGroups[3].Value.Split(", ").ToList(),
        }
    );
}

public record struct RoomExploreState {
    public string Room;
    public int Steps;
}

int DistanceBetweenRooms(string from, string to, Dictionary<string, Room> map) {
    var roomsToExplore = new Queue<RoomExploreState>();
    roomsToExplore.Enqueue(new RoomExploreState { Room = from, Steps = 0});
    while (roomsToExplore.TryDequeue(out var nextExploration)) {
        if (nextExploration.Room == to) {
            return nextExploration.Steps;
        }
        var newStates = map[nextExploration.Room].Tunnels
            .Select(tunnel => new RoomExploreState { Room = tunnel, Steps = nextExploration.Steps + 1});
        
        foreach (var newState in newStates) {
            roomsToExplore.Enqueue(newState);
        }
    }
    throw new InvalidOperationException($"Unreachable room {from} -> {to}");
}

Dictionary<(string, string), int> DistanceBetweenAllRooms(Dictionary<string, Room> map) {
    var distanceMap = new Dictionary<(string, string), int>();
    foreach (var room in map.Keys) {
        foreach (var other in map.Keys) {
            distanceMap.Add((room, other), DistanceBetweenRooms(room, other, map));
        }
    }
    return distanceMap;
}

public record struct State {
    public string CurrentRoom;
    public HashSet<string> OpenValves;
    public int ReleasedPressure;
    public int TimeStep;
    
    public int TotalCurrentFlowRate;
    public int MaxPossibleTotalPressureReleased(int totalTime, int maxFlowRate) => ReleasedPressure + maxFlowRate * (totalTime - TimeStep);
}

IEnumerable<State> NextStates(State currentState, Dictionary<String, Room> map, Dictionary<(string, string), int> distancesBetweenRooms, int totalTime) {
    var currentRoom = currentState.CurrentRoom;
    foreach (var nextTargetRoom in map.Keys) {
        if (currentState.OpenValves.Contains(nextTargetRoom) == false && map[nextTargetRoom].FlowRate > 0) {
            var stepsTaken = distancesBetweenRooms[(currentRoom, nextTargetRoom)] + 1;
            yield return new State {
                CurrentRoom = nextTargetRoom,
                OpenValves = currentState.OpenValves.Append(nextTargetRoom).ToHashSet(),
                ReleasedPressure = currentState.ReleasedPressure + currentState.TotalCurrentFlowRate * stepsTaken,
                TimeStep = currentState.TimeStep + stepsTaken,

                TotalCurrentFlowRate = currentState.TotalCurrentFlowRate + map[nextTargetRoom].FlowRate,
            };
        }
    }
    // Or just wait til the end
    var stepsRemaining = totalTime - currentState.TimeStep;
    yield return currentState with {
        ReleasedPressure = currentState.ReleasedPressure + currentState.TotalCurrentFlowRate * stepsRemaining,
        TimeStep = currentState.TimeStep + stepsRemaining,
    };
}

int MaxPossibleTotalPressureReleased(Dictionary<string, Room> map, Dictionary<(string, string), int> distancesBetweenRooms, State startingState, int totalTime) {
    var statesToExplore = new PriorityQueue<State, int>();
    var maxFlowRate = map.Values.Select(room => room.FlowRate).Sum();
    statesToExplore.Enqueue(startingState, -1 * startingState.MaxPossibleTotalPressureReleased(totalTime, maxFlowRate));
    while (statesToExplore.TryDequeue(out var state, out var p)) {
        if (state.TimeStep == totalTime) {
            return state.ReleasedPressure;
        }
        foreach (var nextState in NextStates(state, map, distancesBetweenRooms, totalTime)) {
            statesToExplore.Enqueue(nextState, -1 * nextState.MaxPossibleTotalPressureReleased(totalTime, maxFlowRate));
        }
    }
    throw new System.InvalidOperationException("Ran out of valid states before reaching total time");
}

public record PartialMovement {
    public string TargetRoom;
    public int StepsAway;
}

public record struct StatePart2 {
    public PartialMovement MyLocation;
    public PartialMovement ElephantLocation;
    public HashSet<string> TargetableValves;
    public int ReleasedPressure;
    public int TimeStep;
    
    public int TotalCurrentFlowRate;
    public int MaxPossibleTotalPressureReleased(int totalTime, int maxFlowRate) => ReleasedPressure + maxFlowRate * (totalTime - TimeStep);
    public int PriorityHeuristic(int totalTime, int maxFlowRate) {
        return ReleasedPressure
            + TotalCurrentFlowRate * (totalTime - TimeStep)
            + (maxFlowRate - TotalCurrentFlowRate) / 2 * (totalTime - TimeStep);
    }

}

IEnumerable<StatePart2> NextStatesPart2(StatePart2 currentState, Dictionary<String, Room> map, Dictionary<(string, string), int> distancesBetweenRooms, int totalTime) {
    // Change my target room if I have reached my destination
    if (currentState.MyLocation.StepsAway == 0) {
        foreach (var nextTargetRoom in currentState.TargetableValves) {
            var stepsAway = distancesBetweenRooms[(currentState.MyLocation.TargetRoom, nextTargetRoom)] + 1;
            yield return currentState with {
                MyLocation = new PartialMovement {
                    TargetRoom = nextTargetRoom,
                    StepsAway = stepsAway,
                },
                TargetableValves = currentState.TargetableValves.Where(room => room != nextTargetRoom).ToHashSet(),
            };
        }
        // Or just wait til end
        yield return currentState with {
            MyLocation = new PartialMovement {
                TargetRoom = currentState.MyLocation.TargetRoom,
                StepsAway = totalTime - currentState.TimeStep,
            }
        };
        yield break;
    }

    // Change elephant friend's target room if they have reached their destination
    if (currentState.ElephantLocation.StepsAway == 0) {
        foreach (var nextTargetRoom in currentState.TargetableValves) {
            var stepsAway = distancesBetweenRooms[(currentState.ElephantLocation.TargetRoom, nextTargetRoom)] + 1;
            yield return currentState with {
                ElephantLocation = new PartialMovement {
                    TargetRoom = nextTargetRoom,
                    StepsAway = stepsAway,
                },
                TargetableValves = currentState.TargetableValves.Where(room => room != nextTargetRoom).ToHashSet(),
            };
        }
        // Or just wait til end
        yield return currentState with {
            ElephantLocation = new PartialMovement {
                TargetRoom = currentState.ElephantLocation.TargetRoom,
                StepsAway = totalTime - currentState.TimeStep,
            }
        };
        yield break;
    }

    // If neither of us have reached our destination, move until one of us has, flip the targeted valve, then go again
    var stepsTaken = Math.Min(currentState.MyLocation.StepsAway, currentState.ElephantLocation.StepsAway);
    var newFlowRate = currentState.TotalCurrentFlowRate;
    if (currentState.MyLocation.StepsAway == stepsTaken) {
        newFlowRate += map[currentState.MyLocation.TargetRoom].FlowRate;
    }
    if (currentState.ElephantLocation.StepsAway == stepsTaken) {
        newFlowRate += map[currentState.ElephantLocation.TargetRoom].FlowRate;
    }
    yield return currentState with {
        MyLocation = new PartialMovement {
            TargetRoom = currentState.MyLocation.TargetRoom,
            StepsAway = currentState.MyLocation.StepsAway - stepsTaken,
        },
        ElephantLocation = new PartialMovement {
            TargetRoom = currentState.ElephantLocation.TargetRoom,
            StepsAway = currentState.ElephantLocation.StepsAway - stepsTaken,
        },
        ReleasedPressure = currentState.ReleasedPressure + currentState.TotalCurrentFlowRate * stepsTaken,
        TimeStep = currentState.TimeStep + stepsTaken,

        TotalCurrentFlowRate = newFlowRate,
    };
}

int MaxPossibleTotalPressureReleasedPart2(Dictionary<string, Room> map, Dictionary<(string, string), int> distancesBetweenRooms, StatePart2 startingState, int totalTime, int maxQueueSize) {
    var statesToExplore = new PriorityQueue<StatePart2, int>();
    var maxFlowRate = map.Values.Select(room => room.FlowRate).Sum();
    var lowerBound = 0;

    statesToExplore.Enqueue(startingState, -1 * startingState.PriorityHeuristic(totalTime, maxFlowRate));
    while (statesToExplore.TryDequeue(out var state, out var _)) {
        if (state.MaxPossibleTotalPressureReleased(totalTime, maxFlowRate) <= lowerBound) {
            continue;
        }
        // Trim the queue if it gets too large
        // Super ugly, and it seems a bit unstable, especially with low queue sizes but it kinda works
        if (statesToExplore.Count > maxQueueSize) {
            Console.WriteLine("Trimming PQ");
            var newQueue = new PriorityQueue<StatePart2, int>();
            for (var i = 0; i < maxQueueSize * 0.8; i++) {
                var result = statesToExplore.Dequeue();
                newQueue.Enqueue(result, result.PriorityHeuristic(totalTime, maxFlowRate));
            }
            statesToExplore.Clear();
            statesToExplore = newQueue;
        }
        if (state.TimeStep == totalTime) {
            if (state.ReleasedPressure > lowerBound) {
                lowerBound = state.ReleasedPressure;
                Console.WriteLine($"New lower bound {lowerBound}");
            }
        }
        foreach (var nextState in NextStatesPart2(state, map, distancesBetweenRooms, totalTime)) {
            statesToExplore.Enqueue(nextState, -1 * nextState.PriorityHeuristic(totalTime, maxFlowRate));
        }
    }
    return lowerBound;
}

In [6]:
var input = System.IO.File.ReadAllLines("input.txt");
input.Take(10)

index,value
0,"Valve VN has flow rate=0; tunnels lead to valves LW, TK"
1,"Valve FQ has flow rate=0; tunnels lead to valves AJ, YC"
2,"Valve DO has flow rate=0; tunnels lead to valves RV, HJ"
3,"Valve MW has flow rate=0; tunnels lead to valves TE, HJ"
4,"Valve LT has flow rate=5; tunnels lead to valves KO, SG, KH, HZ, RV"
5,"Valve UJ has flow rate=0; tunnels lead to valves FW, DE"
6,"Valve IZ has flow rate=0; tunnels lead to valves LU, SX"
7,"Valve FE has flow rate=17; tunnels lead to valves WG, WI, LC"
8,"Valve KS has flow rate=25; tunnels lead to valves QA, BT"
9,"Valve HJ has flow rate=11; tunnels lead to valves MW, CZ, ZE, DO"


In [7]:
var map = input.Select(ParseRoom)
    .ToDictionary(room => room.Item1, room => room.Item2);

var startingState = new State {
    CurrentRoom = "AA",
    OpenValves = new HashSet<string>(),
    ReleasedPressure = 0,
    TimeStep = 0,
    TotalCurrentFlowRate = 0,
};

var distancesBetweenRooms = DistanceBetweenAllRooms(map);
MaxPossibleTotalPressureReleased(map, distancesBetweenRooms, startingState, 30)

1915

In [10]:
var map = input.Select(ParseRoom)
    .ToDictionary(room => room.Item1, room => room.Item2);

var startingState = new StatePart2 {
    MyLocation = new PartialMovement {
        TargetRoom = "AA",
        StepsAway = 0,
    },
    ElephantLocation = new PartialMovement {
        TargetRoom = "AA",
        StepsAway = 0,
    },
    TargetableValves = map.Keys.Where(room => map[room].FlowRate > 0).ToHashSet(),
    ReleasedPressure = 0,
    TimeStep = 0,
    TotalCurrentFlowRate = 0,
};

var distancesBetweenRooms = DistanceBetweenAllRooms(map);

MaxPossibleTotalPressureReleasedPart2(map, distancesBetweenRooms, startingState, 26, 1_000_000)

New lower bound 2762
New lower bound 2772
Trimming PQ


2772